In [6]:
!cat .env

STAGE=PRODUCTION


In [1]:
import json
from django.contrib.admin.utils import flatten
from helpers.strings import keys_to_snake_case, keys_to_camel_case
from rest_framework_json_api.utils import format_field_names
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
def generate_select_multiple_export(field):

    result={"cat":[]}    
    values=field["selectOptions"]["options"]

    for value in values:
        if field["apiName"] == "demandOrigin":
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + " ", ""]})
        else:
            result["cat"].append({"if":[{"in": 
                                     [value["value"], {"var": "formData."+field["apiName"]}]
                                    }, value["name"] + ", ", ""]})


    return result

In [3]:
def generate_select_export(field):
    result={"if":[]}    
    values=field["selectOptions"]["options"]
    
    for value in values:
        result["if"].append({"==":[{"var": "formData."+field["apiName"]}, value["value"]]})
        result["if"].append(value["name"])
        
    return result

In [4]:
def generate_export(field):
    result={"header": field["displayName"], "key": field["apiName"]}
    
    if field["dataType"]=="timestamp":
        result["isDate"]=True
#     elif (field["dataType"]=="select") and ("options" in field["selectOptions"]):
#         result["isSelect"]=True
#     elif (field["dataType"]=="selectMultiple") and ("options" in field["selectOptions"]):
    elif field["dataType"]=="select" or field["dataType"]=="selectMultiple":
        result["isSelect"]=True
    elif field["dataType"]=="jsonLogic":
        result["logic"]=field["logic"]
    elif field["dataType"]=="float":
        result["style"]= {
                "num_fmt": "0.000"
                }
#     elif field["dataType"]=="boolean":
#         result["logic"] = {
#                             "if": [{
#                                 "===": [{
#                                     "var": "formData." + field["apiName"]
#                                 }, True]
#                             }, "Sim", "Não"]
#                         }
    
    return result

In [5]:
def generate_array_export(field, number=8):
    result_list=defaultdict(list)
    values=field["innerFields"]
    
    for value in values:
        
        if (value["dataType"] == "select") and ("options" in value["selectOptions"]):            
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["if"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["if"].append({"==":[{"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}, value_select["value"]]})
                    result["logic"]["if"].append(value_select["name"])

                result_list[index].append(result)
        
        elif (value["dataType"] == "boolean") and ("display" in value["apiName"]):
            pass
        
        elif value["dataType"] == "innerImagesArray":
            pass
                
        elif (value["dataType"] == "selectMultiple") and ("options" in value["selectOptions"]):
            for index in range(1, number+1):            
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " "  + str(index)
                result["key"]=value["apiName"]+str(index)
                result["logic"]={}
                result["logic"]["cat"]=[]
                values_select=value["selectOptions"]["options"]

                for value_select in values_select:
                    result["logic"]["cat"].append({"if":[{"in": 
                                     [value_select["value"], {"var": "formData." + field["apiName"]+ "."+str(index-1) + "." +value["apiName"]}]
                                    }, value_select["name"], ", "]})

                result_list[index].append(result)
            
        else:
            for index in range(1, number+1):
                result={}
                result["header"]=field["displayName"] + " - " + value["displayName"] + " " + str(index)
                result["key"]=value["apiName"]+str(index)
                
#                 if value["dataType"] == "boolean":
#                     result["logic"] = {
#                                         "if": [{
#                                             "===": [{
#                                                 "var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]
#                                             }, True]
#                                         }, "Sim", "Não"]
#                                     }
#                 else:
#                     result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]}
                
                
                result["logic"]= {"var": "formData." + field["apiName"] + "." + str(index-1) + "." + value["apiName"]}
                if value["dataType"] == "timestamp":
                    result["isDate"] = True
                result_list[index].append(result)

#     print(result_list[1])
    new_result_list = [v for k, v in sorted(result_list.items(), key=lambda item: item[0])]
    return new_result_list

In [6]:
def generate_array_export_v2(field, number=5):
    result_list={"key": field["apiName"], "header": field["displayName"], "isArray": True, "maxRepetitions": number, "fields":[]}
    values=field["innerFields"]
    
    for value in values:        
        if (value["dataType"] == "boolean") and ("display" in value["apiName"]):
            continue
                
        result_dict={"field": value["apiName"], "header": value["displayName"]}
        if value["dataType"] == "select" or value["dataType"] == "select":
            result_dict["isSelect"] = True
        if value["dataType"] == "timestamp":
            result_dict["isDate"] = True
        if value["dataType"] == "innerImagesArray":
            result_dict["isImage"] = True
            result_dict["repeatInTheEnd"] = True
        result_list["fields"].append(result_dict)
#         print(result_list)
    
    return result_list

In [7]:
def update_export_data(data, occ=None, filename=None):
    if occ is not None:
        if 'measurement_columns' in occ.form_fields.keys():
            del occ.form_fields['measurement_columns']
        occ.form_fields['measurementColumns'] = data
        occ.save()
    if filename is not None:
        with open(filename + '.txt', 'w') as outfile:
            json.dump(data, outfile, indent=4)

In [8]:
def format_keys(obj, format_type="camelize"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(value, dict):
                obj[key] = format_field_names(
                            format_keys(value), format_type
                            )
            elif isinstance(value, list):
                list_temp = value.copy()
                for i, item in enumerate(value):
                    if isinstance(item, dict):
                        list_temp[i] = format_field_names(
                        format_keys(item), format_type
                        )
                    else:
                        list_temp[i] = item
                obj[key] = list_temp
    return obj

In [9]:
company=Company.objects.get(name='Arteris Intervias')
company

<Company: b8450161-14d1-4de8-b254-7d1ddc4a9c20: Arteris Intervias>

In [10]:
# occ=OccurrenceType.objects.get(uuid='f6033d7d-9649-420d-9125-2000aefcd25a')
occ=OccurrenceType.objects.filter(company=company).order_by('name')
occ.count()

314

In [11]:
for match in tqdm(occ):
    for index, field in enumerate(match.form_fields['fields']):
        match.form_fields['fields'][index]=format_keys(keys_to_camel_case(field))
    match.save()

  0%|          | 0/314 [00:00<?, ?it/s]

In [12]:
for match in tqdm(occ):
        
    try:
        columns=[]

        for field in match.form_fields['fields']:
            if field['dataType'] != 'arrayOfObjects':
                columns.append(generate_export(field))
            else:
                columns.append(generate_array_export_v2(field, 5))


        columns=flatten(flatten(columns))
        update_export_data(columns, occ=match)
    
    except Exception as e:
        print(match.uuid)
        print(f'{match.uuidd} - {e}')

  0%|          | 0/314 [00:00<?, ?it/s]